In [1]:
import json
from re import L
from tqdm import tqdm
import os
import numpy as np
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
torch.backends.cudnn.benchmark = True
from sklearn.model_selection import KFold
from tensorboardX import SummaryWriter
from model_cnnlstm import Multifusion
import math as m
from scipy.stats import pearsonr


In [36]:
m1 = Multifusion(device='cpu')

In [37]:
m1.encoder.one_cnn.load_state_dict(d1)

<All keys matched successfully>

In [6]:
from model_cnnlstm import Encoder
model = Encoder()

In [17]:
d1 = torch.load('/home/yangjunran/soil_moisture/feature_cnn/data/feature_test.bin')

In [12]:
d = model.one_cnn.state_dict()

In [20]:
d2 = torch.load('/home/yangjunran/soil_moisture/feature_cnn/data/linear_test.bin')

In [18]:
d1.keys()

odict_keys(['0.weight', '0.bias', '2.weight', '2.bias', '4.weight', '4.bias'])

In [21]:
model.feature_linear.load_state_dict(d2)

<All keys matched successfully>

In [19]:
model.one_cnn.load_state_dict(d1,strict=False)

<All keys matched successfully>

In [2]:

torch.set_default_tensor_type(torch.DoubleTensor)





device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cpu=torch.device('cpu')






class Engine(object):
	"""Engine that runs training and inference.
	Args
		- cur_epoch (int): Current epoch.
		- print_every (int): How frequently (# batches) to print loss.
		- validate_every (int): How frequently (# epochs) to run validation.
		
	"""

	def __init__(self, cpu, device,  cur_epoch=0, cur_iter=0, ):
		self.cur_epoch = cur_epoch
		self.cur_iter = cur_iter
		self.bestval_epoch = cur_epoch
		self.train_loss = []
		self.val_loss = []
		self.bestval = 1e10
		self.device = device
		self.cpu=cpu
		self.cc=0.
		self.rb=0.
		self.rmse=0.
		self.mae=0.


	def train(self):
		loss_epoch = 0
		num_batches = 0
		model.train()
		model.encoder.feature_linear.eval()
		model.encoder.one_cnn.eval()
		model.float()
		# Train loop
		for (tp,dem,skt,sw,label) in tqdm(zip(tpdata_train, demdata_train, sktdata_train, swdata_train, labeldata_train)):
			tp,dem,skt,sw,label=tp.to(self.device),dem.to(self.device),skt.to(self.device),sw.to(self.device),label.view(-1,1).to(self.device)
			# efficiently zero gradients
			# for p in model.parameters():
			# 	p.grad = None
			model.zero_grad()
			bz = tp.shape[0]
			l=tp.shape[1]
			sw_center=sw[:,:,:,3,3].view(bz*l,-1).to(self.device)
			skt_center=skt[:,:,:,3,3].view(bz*l,-1).to(self.device)

			pred = model(tp,dem,skt,sw,sw_center,skt_center)
			# pred = tp[:,10,:,3,3].view(bz,-1)
			
			# loss =(F.mse_loss(pred, label, reduction='none') * torch.sigmoid(F.mse_loss(pred, label, reduction='none'))).mean()
			
			loss = (F.mse_loss(pred.float(), label.float(), reduction='none')).mean()
		
			
			loss.backward()
			loss_epoch += float(loss.item())

			num_batches += 1
			optimizer.step()

			writer.add_scalar('train_loss', loss.item(), self.cur_iter)
			self.cur_iter += 1
		
		
		loss_epoch = loss_epoch / num_batches
		self.train_loss.append(loss_epoch)
		self.cur_epoch += 1

	def validate(self):
		model.eval()
		model.float()

		with torch.no_grad():	
			num_batches = 0
			wp_epoch = 0.
			self.rmse = 0.
			self.rb = 0.
			self.mae=0.
			self.cc=0.
			real_list=[]
			pred_list=[]

			# Validation loop
			for batch_num, (tp,dem,skt,sw,label) in  enumerate((zip(tpdata_test,demdata_test,sktdata_test,swdata_test,labeldata_test)),0):
				
				tp,dem,skt,sw,label=tp.to(self.device),dem.to(self.device),skt.to(self.device),sw.to(self.device),label.view(-1,1).to(self.device)
				

				bz = tp.shape[0]
				l=tp.shape[1]
				sw_center=sw[:,:,:,3,3].view(bz*l,-1).to(self.device)
				skt_center=skt[:,:,:,3,3].view(bz*l,-1).to(self.device)

				pred = model(tp,dem,skt,sw,sw_center,skt_center)

				wp_epoch += float(F.mse_loss(pred, label, reduction='mean'))
				self.rmse += float(F.mse_loss(pred, label, reduction='sum'))
				self.mae += float(F.l1_loss(pred, label, reduction='mean'))
				pred_list.extend(pred.to(self.cpu).view(-1).numpy())
				real_list.extend(label.to(self.cpu).view(-1).numpy())


				num_batches += 1
			self.rmse=m.sqrt(self.rmse/(num_batches*batchsize*13))
			wp_loss = wp_epoch / float(num_batches)
			self.mae = self.mae / float(num_batches)
			self.cc,p=pearsonr(pred_list,real_list)
			self.rb=(sum(pred_list)-sum(real_list))/sum(real_list)

			tqdm.write(f'Epoch {self.cur_epoch:03d}, Batch {batch_num:03d}:' + f' Wp: {wp_loss:3.3f}')

			writer.add_scalar('val_loss', wp_loss, self.cur_epoch)
			writer.add_scalar('rmse', self.rmse, self.cur_epoch)
			writer.add_scalar('mae', self.mae, self.cur_epoch)
			writer.add_scalar('cc', self.cc, self.cur_epoch)
			writer.add_scalar('rb', self.rb, self.cur_epoch)
			# scheduler.step(wp_loss)
			self.val_loss.append(wp_loss)
	

	def save(self):

		save_best = False
		if self.val_loss[-1] <= self.bestval:
			self.bestval = self.val_loss[-1]
			self.bestval_epoch = self.cur_epoch
			save_best = True
		
		# Create a dictionary of all data to save
		log_table = {
			'epoch': self.cur_epoch,
			'iter': self.cur_iter,
			'bestval': self.bestval,
			'bestval_epoch': self.bestval_epoch,
			'train_loss': self.train_loss,
			'val_loss': self.val_loss,
			'cc': self.cc,
			'mae': self.mae,
			'mse': self.rmse,
			'rb': self.rb
		}

		# Save ckpt for every epoch
		torch.save(model.state_dict(), os.path.join(logdir, 'model_%d.pth'%self.cur_epoch))

		# Save the recent model/optimizer states
		torch.save(model.state_dict(), os.path.join(logdir, 'model.pth'))
		torch.save(optimizer.state_dict(), os.path.join(logdir, 'recent_optim.pth'))

		# Log other data corresponding to the recent model
		with open(os.path.join(logdir, 'recent.log'), 'w') as f:
			f.write(json.dumps(log_table))

		tqdm.write('====== Saved recent model ======>')
		
		if save_best:
			torch.save(model.state_dict(), os.path.join(logdir, 'best_model.pth'))
			torch.save(optimizer.state_dict(), os.path.join(logdir, 'best_optim.pth'))
			tqdm.write('====== Overwrote best model ======>')
            

            


In [4]:

###settings

num_epochs = 200
val_every =1
batchsize=512
timestep=14
###

###model
# writer = SummaryWriter(comment='local')
model = model=Multifusion(device)
one_cnn_dict = torch.load('/home/yangjunran/soil_moisture/feature_cnn/data/feature_test.bin')
model.encoder.one_cnn.load_state_dict(one_cnn_dict)
feature_linear_dict = torch.load('/home/yangjunran/soil_moisture/feature_cnn/data/linear_test.bin')
model.encoder.feature_linear.load_state_dict(feature_linear_dict)
# optimizer = optim.SGD(model.parameters(),lr=1e-5)
# torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=3, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=False)
# trainer = Engine(cpu,device)

###
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print ('Total trainable parameters: ', params)

Total trainable parameters:  63763


In [5]:

###data

tp_origin=np.load("/home/yangjunran/soil_moisture/normalize/tp.npy")
dem_origin=np.load("/home/yangjunran/soil_moisture/normalize/dem.npy")
skt_origin=np.load("/home/yangjunran/soil_moisture/normalize/skt.npy")
sw_origin=np.load("/home/yangjunran/soil_moisture/normalize/sw.npy")

label_origin =np.load("/home/yangjunran/soil_moisture/label.npy")



num_k=0

kf = KFold(n_splits=8)
for train_index, test_index in kf.split(tp_origin):
	print("TRAIN:", train_index, "TEST:", test_index)
	tp_train, tp_test = tp_origin[train_index].reshape((-1,timestep,1,7,7)), tp_origin[test_index].reshape((-1,timestep,1,7,7))
	dem_train, dem_test = dem_origin[train_index].reshape((-1,timestep,1,7,7)), dem_origin[test_index].reshape((-1,timestep,1,7,7))
	skt_train, skt_test = skt_origin[train_index].reshape((-1,timestep,1,7,7)), skt_origin[test_index].reshape((-1,timestep,1,7,7))
	sw_train, sw_test = sw_origin[train_index].reshape((-1,timestep,1,7,7)), sw_origin[test_index].reshape((-1,timestep,1,7,7))

	label_train, label_test = label_origin[train_index].reshape((-1,1,1,1,1)), label_origin[test_index].reshape((-1,1,1,1,1))

	tpdata_train=DataLoader(tp_train, batch_size=batchsize, shuffle=False, sampler=None,
				batch_sampler=None, num_workers=0, collate_fn=None,
				pin_memory=False, drop_last=True, timeout=0,
				worker_init_fn=None, prefetch_factor=2,
				persistent_workers=False)
	tpdata_test=DataLoader(tp_test, batch_size=batchsize, shuffle=False, sampler=None,
				batch_sampler=None, num_workers=0, collate_fn=None,
				pin_memory=False, drop_last=True, timeout=0,
				worker_init_fn=None, prefetch_factor=2,
				persistent_workers=False)

	demdata_train=DataLoader(dem_train, batch_size=batchsize, shuffle=False, sampler=None,
				batch_sampler=None, num_workers=0, collate_fn=None,
				pin_memory=False, drop_last=True, timeout=0,
				worker_init_fn=None, prefetch_factor=2,
				persistent_workers=False)
	demdata_test=DataLoader(dem_test, batch_size=batchsize, shuffle=False, sampler=None,
				batch_sampler=None, num_workers=0, collate_fn=None,
				pin_memory=False, drop_last=True, timeout=0,
				worker_init_fn=None, prefetch_factor=2,
				persistent_workers=False)

	sktdata_train=DataLoader(skt_train, batch_size=batchsize, shuffle=False, sampler=None,
				batch_sampler=None, num_workers=0, collate_fn=None,
				pin_memory=False, drop_last=True, timeout=0,
				worker_init_fn=None, prefetch_factor=2,
				persistent_workers=False)
	sktdata_test=DataLoader(skt_test, batch_size=batchsize, shuffle=False, sampler=None,
				batch_sampler=None, num_workers=0, collate_fn=None,
				pin_memory=False, drop_last=True, timeout=0,
				worker_init_fn=None, prefetch_factor=2,
				persistent_workers=False)

	swdata_train=DataLoader(sw_train, batch_size=batchsize, shuffle=False, sampler=None,
				batch_sampler=None, num_workers=0, collate_fn=None,
				pin_memory=False, drop_last=True, timeout=0,
				worker_init_fn=None, prefetch_factor=2,
				persistent_workers=False)
	swdata_test=DataLoader(sw_test, batch_size=batchsize, shuffle=False, sampler=None,
				batch_sampler=None, num_workers=0, collate_fn=None,
				pin_memory=False, drop_last=True, timeout=0,
				worker_init_fn=None, prefetch_factor=2,
				persistent_workers=False)
	
	labeldata_train=DataLoader(label_train, batch_size=batchsize, shuffle=False, sampler=None,
				batch_sampler=None, num_workers=0, collate_fn=None,
				pin_memory=False, drop_last=True, timeout=0,
				worker_init_fn=None, prefetch_factor=2,
				persistent_workers=False)
	labeldata_test=DataLoader(label_test, batch_size=batchsize, shuffle=False, sampler=None,
				batch_sampler=None, num_workers=0, collate_fn=None,
				pin_memory=False, drop_last=True, timeout=0,
				worker_init_fn=None, prefetch_factor=2,
				persistent_workers=False)
	num_k=num_k+1
	break


TRAIN: [ 42014  42015  42016 ... 336103 336104 336105] TEST: [    0     1     2 ... 42011 42012 42013]


In [5]:
trainer = Engine(cpu,device)
    

In [6]:
for tqdm()

Epoch 000, Batch 033: Wp: 0.013


NameError: name 'writer' is not defined

In [12]:
t2 = tp_origin[17728]
t2.shape

(14, 7, 7)

In [ ]:

	logdir='/home/yangjunran/soil_moisture/log'+str(num_k)
	tail='cnnlstm21o'+str(num_k)
	writer = SummaryWriter(comment=tail)
	model = model=Multifusion(device)
	# optimizer = optim.Adam(model.parameters(),lr=5*1e-5)
	optimizer = optim.Adam(model.parameters(),lr=5*1e-5)

	# scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=3, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=False)
	trainer = Engine(cpu,device)
	# Load checkpoint
	if not os.path.isdir(logdir):
		os.makedirs(logdir)
		print('Created dir:',logdir)
	elif os.path.isfile(os.path.join(logdir, 'recent.log')):
		print('Loading checkpoint from ' + logdir)
		with open(os.path.join(logdir, 'recent.log'), 'r') as f:
			log_table = json.load(f)
		# Load variables
		trainer.cur_epoch = log_table['epoch']
		if 'iter' in log_table: trainer.cur_iter = log_table['iter']
		trainer.bestval = log_table['bestval']
		trainer.train_loss = log_table['train_loss']
		trainer.val_loss = log_table['val_loss']
		# Load checkpoint
		model.load_state_dict(torch.load(os.path.join(logdir, 'model.pth')))
		optimizer.load_state_dict(torch.load(os.path.join(logdir, 'recent_optim.pth')))
	


	# tail='cnnlstm21o'+str(num_k)
	# writer = SummaryWriter(comment=tail)
	# model = model=Multifusion(device)
	# optimizer = optim.Adam(model.parameters(),lr=5*1e-5)
	# # scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=3, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=False)
	# trainer = Engine(cpu,device)


	

	###
	model_parameters = filter(lambda p: p.requires_grad, model.parameters())
	params = sum([np.prod(p.size()) for p in model_parameters])
	print ('Total trainable parameters: ', params)

	for epoch in tqdm(range(trainer.cur_epoch, num_epochs)): 
		trainer.train()
		if epoch % val_every == 0: 
			trainer.validate()
			trainer.save()

###      















In [22]:
import numpy as np

In [23]:
dem = np.load('/home/yangjunran/soil_moisture/dem.npy')

In [25]:
dem1 = dem.reshape(-1,7,7)

In [32]:
for i in dem1:
    print(i.shape)
    break

(7, 7)


In [35]:
for i in model.parameters():
    print(i)

Parameter containing:
tensor([[[[ 0.0631, -0.1425,  0.0939, -0.0577,  0.0290],
          [-0.0959,  0.1298,  0.0771,  0.1217,  0.1469],
          [-0.0409,  0.1098, -0.0906,  0.1518, -0.1701],
          [-0.0272,  0.0936,  0.1997,  0.1224, -0.1682],
          [ 0.0577,  0.1365, -0.1405, -0.1163,  0.0578]]],


        [[[ 0.1666,  0.0560,  0.1373,  0.0385,  0.0657],
          [ 0.0371,  0.0604,  0.0040,  0.1380,  0.0202],
          [-0.0180, -0.1738,  0.1769, -0.1558,  0.1509],
          [ 0.1508, -0.0876, -0.1045, -0.1446,  0.0522],
          [ 0.1004,  0.0853, -0.0186,  0.1580, -0.1897]]],


        [[[-0.1577,  0.1172,  0.1202,  0.0364, -0.0406],
          [ 0.0244,  0.0887, -0.0980,  0.1328, -0.0393],
          [-0.0359,  0.1735, -0.0965,  0.0998, -0.1817],
          [ 0.1211, -0.0994,  0.0154, -0.1811, -0.0077],
          [ 0.1150,  0.0641, -0.1593,  0.0040, -0.0825]]],


        [[[ 0.1353, -0.1575, -0.0544, -0.1349, -0.1732],
          [ 0.0547,  0.0596, -0.0311,  0.1874,  0.0780

In [29]:
dvars = ['dem.npy','skt.npy','sw.npy','tp.npy']

In [33]:
basedir = '/home/yangjunran/soil_moisture/'
normdir = '/home/yangjunran/soil_moisture/normalize/'
for i in dvars:
    data = np.load(basedir+i)
    data1 = data.reshape(-1,7,7)
    for j in range(len(data1)):
        pic = data1[j]
        mean = pic.mean()
        std = pic.std()
        if std == 0:
            continue
        data1[j] = (pic-mean)/std
    data1 = data1.reshape(data.shape)
    np.save(normdir+i,data1)
            
    

In [6]:
model.load_state_dict(torch.load('/home/yangjunran/soil_moisture/log2/best_model.pth'))

<All keys matched successfully>

In [10]:
model.float()

Multifusion(
  (encoder): Encoder(
    (sw_cnn): Sequential(
      (0): Conv2d(1, 8, kernel_size=(5, 5), stride=(1, 1), padding=same)
      (1): ReLU(inplace=True)
      (2): Dropout(p=0.1, inplace=False)
      (3): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (4): ReLU(inplace=True)
      (5): Conv2d(8, 4, kernel_size=(2, 2), stride=(1, 1), padding=same)
      (6): ReLU(inplace=True)
      (7): AdaptiveMaxPool2d(output_size=(3, 3))
      (8): Flatten(start_dim=1, end_dim=-1)
    )
    (tp_cnn): Sequential(
      (0): Conv2d(1, 8, kernel_size=(5, 5), stride=(1, 1), padding=same)
      (1): ReLU(inplace=True)
      (2): Dropout(p=0.1, inplace=False)
      (3): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (4): ReLU(inplace=True)
      (5): Conv2d(8, 4, kernel_size=(2, 2), stride=(1, 1), padding=same)
      (6): ReLU(inplace=True)
      (7): AdaptiveMaxPool2d(output_size=(3, 3))
      (8): Flatten(start_dim=1, end_dim=-1)
    )
    (dem_cnn): 

In [11]:
with torch.no_grad():	
    num_batches = 0
    wp_epoch = 0.
    rmse = 0.
    rb = 0.
    mae=0.
    cc=0.
    real_list=[]
    pred_list=[]
    for batch_num, (tp,dem,skt,sw,label) in  enumerate((zip(tpdata_test,demdata_test,sktdata_test,swdata_test,labeldata_test)),0):
        
        tp,dem,skt,sw,label=tp.to(device),dem.to(device),skt.to(device),sw.to(device),label.view(-1,1).to(device)

        bz = tp.shape[0]
        l=tp.shape[1]
        sw_center=sw[:,:,:,3,3].view(bz*l,-1).to(device)
        skt_center=skt[:,:,:,3,3].view(bz*l,-1).to(device)

        pred = model(tp,dem,skt,sw,sw_center,skt_center)
        print(pred)

        wp_epoch += float(F.mse_loss(pred, label, reduction='mean'))
        rmse += float(F.mse_loss(pred, label, reduction='sum'))
        mae += float(F.l1_loss(pred, label, reduction='mean'))
        pred_list.extend(pred.to(cpu).view(-1).numpy())
        real_list.extend(label.to(cpu).view(-1).numpy())


        num_batches += 1
rmse=m.sqrt(rmse/(num_batches*batchsize*13))
wp_loss = wp_epoch / float(num_batches)

/home/yangjunran/anaconda3/envs/py38/lib/python3.8/site-packages/torch/nn/modules/conv.py:459: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /opt/conda/conda-bld/pytorch_1666642975312/work/aten/src/ATen/native/Convolution.cpp:895.)
  return F.conv2d(input, weight, bias, self.stride,


tensor([[0.1624],
        [0.1621],
        [0.1716],
        [0.1779],
        [0.1774],
        [0.1794],
        [0.1829],
        [0.1582],
        [0.1761],
        [0.1815],
        [0.1735],
        [0.1589],
        [0.1375],
        [0.1457],
        [0.1630],
        [0.1150],
        [0.1271],
        [0.1258],
        [0.1185],
        [0.1368],
        [0.1443],
        [0.1485],
        [0.1407],
        [0.1471],
        [0.1451],
        [0.1343],
        [0.1290],
        [0.1327],
        [0.1516],
        [0.1170],
        [0.1660],
        [0.1788],
        [0.1499],
        [0.1265],
        [0.1395],
        [0.1119],
        [0.1051],
        [0.1173],
        [0.1678],
        [0.1935],
        [0.1837],
        [0.1464],
        [0.1430],
        [0.1538],
        [0.1644],
        [0.1382],
        [0.1422],
        [0.1238],
        [0.1271],
        [0.1615],
        [0.1424],
        [0.1169],
        [0.2127],
        [0.2696],
        [0.2522],
        [0

In [9]:
num_batches

0

# Get best result

In [43]:
from new_train import Engine,cnnlstm_Dataset,inner_Dataset
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from model_cnnlstm import Multifusion
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
from scipy.stats import pearsonr
import math as m


In [11]:
basedir = '/share/yangjunran/soil_moisture_data/time30_origin_pair'
tp_origin = np.load(basedir+"/normalize/tp.npy",allow_pickle=True)
skt_origin = np.load(basedir+'/normalize/skt.npy',allow_pickle=True)
sw_origin = np.load(basedir+'/normalize/sw.npy',allow_pickle=True)
dem_origin = np.load(basedir+'/normalize/dem.npy',allow_pickle=True)
label_origin = np.load(basedir+"/normalize/label.npy",allow_pickle=True)


In [18]:
device = torch.device('cuda:1')
cpu = torch.device('cpu')

In [47]:
class Validate():
    def __init__(self) -> None:
        self.sample_num = 0

        self.rmse = 0.
        self.rb = 0.
        self.mae = 0.
        self.device = torch.device('cuda:1')
        self.timestep = 30
        self.cc = 0.
        self.cpu = torch.device('cpu')
    def validate(self):
        model.eval()
        model.float()
        with torch.no_grad():
            self.sample_num = 0
            wp_epoch = 0.
            self.rmse = 0.
            self.rb = 0.
            self.mae = 0.
            self.cc = 0.
            self.cc_list=[]
            real_list = []
            pred_list = []
            all_batch = 0
            self.points_num=len(val_points)
            for point in val_points:
                point_pred_list,point_real_list = [],[]
                inner_dataset = inner_Dataset(point,timestep=self.timestep)
                val_dataloader = DataLoader(inner_dataset, batch_size=512, shuffle=False, sampler=None,
                                    batch_sampler=None, num_workers=0, collate_fn=None,
                                    pin_memory=False, drop_last=False, timeout=0,
                                    worker_init_fn=None, prefetch_factor=2,
                                    persistent_workers=False)
            # Validation loop
                num_batches = 0
                self.sample_num += len(inner_dataset)
                for batch_num, (tp, dem, skt, sw, label) in enumerate(val_dataloader):
                    # print(len(tp))

                    tp, dem, skt, sw, label = tp.to(self.device), dem.to(self.device), skt.to(
                        self.device), sw.to(self.device), label.view(-1, 1).to(self.device)

                    bz = tp.shape[0]
                    l = tp.shape[1]
                    sw_center = sw[:, :, :, 3, 3].view(bz*l, -1).to(self.device)
                    skt_center = skt[:, :, :, 3, 3].view(bz*l, -1).to(self.device)

                    pred = model(tp, dem, skt, sw, sw_center, skt_center)
                    # print(len(pred.to(self.cpu).view(-1)))
                    # print(len(label.to(self.cpu).view(-1)))

                    wp_epoch += float(F.mse_loss(pred, label, reduction='mean'))
                    self.rmse += float(F.mse_loss(pred, label, reduction='sum'))
                    # self.mae += float(F.l1_loss(pred, label, reduction='mean'))
                    self.mae += float(F.l1_loss(pred, label, reduction='sum'))
                    point_pred_list.extend(pred.to(self.cpu).view(-1).numpy())
                    point_real_list.extend(label.to(self.cpu).view(-1).numpy())
                    pred_list.extend(point_pred_list)
                    real_list.extend(point_real_list)

                    # era5_list.extend(sw_center.to(self.cpu).view(-1).numpy())

                    num_batches += 1
                all_batch+=num_batches
                # print('ppl:',len(point_pred_list))
                cc, p = pearsonr(point_pred_list, point_real_list)
                self.cc_list.append(cc)
            self.rmse = m.sqrt(self.rmse/self.sample_num)
            self.cc = np.mean(self.cc_list)
            wp_loss = wp_epoch / float(all_batch)
            self.mae = self.mae / float(self.sample_num)

            # base_cc,p = pearsonr(era5_list,real_list)
            self.rb = (sum(pred_list)-sum(real_list))/sum(real_list)


            # writer.add_scalar('base_cc', 0.374012408672994, self.cur_epoch)

            # scheduler.step(wp_loss)



In [49]:
num_k=0
batchsize=512
timestep=30
kf = KFold(n_splits=8,shuffle=True,random_state=1)
best_val_situ = pd.DataFrame(columns=['round','cc','mae','rmse','rb'])
for train_index, test_index in kf.split(tp_origin):
    print("TRAIN:", train_index, "TEST:", test_index)
    # np.random.shuffle(train_index)
    # np.random.shuffle(test_index)
    # print("TRAIN:", train_index, "TEST:", test_index)
    
    tp_train, tp_val = tp_origin[train_index], tp_origin[test_index]
    dem_train, dem_val = dem_origin[train_index], dem_origin[test_index]
    skt_train, skt_val = skt_origin[train_index], skt_origin[test_index]
    sw_train, sw_val = sw_origin[train_index], sw_origin[test_index]
    label_train, label_val = label_origin[train_index], label_origin[test_index]
    # print(tp_train.shape)


    val_points = cnnlstm_Dataset(tp_val,skt_val,sw_val,dem_val,label_val)
    # val_dataloader = DataLoader(val_dataset,batch_size=batchsize, shuffle=False, sampler=None,
    #                             batch_sampler=None, num_workers=0, collate_fn=None,
    #                             pin_memory=False, drop_last=True, timeout=0,
    #                             worker_init_fn=None, prefetch_factor=2,
    #                             persistent_workers=False)
    num_k=num_k+1

    logdir='/home/yangjunran/soil_moisture/k=8,t=30/'+str(num_k)
    model = model=Multifusion(device)
    best_dir = logdir+'/best_model.pth'

    # trainer = Engine(cpu,device,batchsize=batchsize,timestep=timestep)
    # Load checkpoint
        # Load checkpoint
    model.load_state_dict(torch.load(best_dir))
    trainer = Validate()
    trainer.validate()
    best_val_situ.loc[len(best_val_situ)] = [num_k,trainer.cc,trainer.mae,trainer.rmse,trainer.rb]
print(best_val_situ)

TRAIN: [  0   1   2   3   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  28  29  30  31  32  33  35  36  37  38
  40  41  42  43  45  46  47  48  49  50  51  52  53  54  55  56  57  58
  59  60  61  62  63  64  65  66  68  70  71  72  73  74  75  76  77  78
  79  80  81  82  83  84  85  86  87  90  91  92  94  96  97  98 100 101
 102 103 104 105 106 108 109 110 111 112 113 114 115 117 118 119 121 122
 124 125 126 127 128 129 131 132 133 134 135 136 137 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 173 174 175 176 177 178 179 181
 182 183 184 185 186 187 188 189 191 192 193 194 195 196 197 198 199 200
 202 203 204 206 207 208 209 210 212 213 214 215 218 219 220 222 223 224
 225 226 227 228 230 231 232 234] TEST: [  4  27  34  39  44  67  69  88  89  93  95  99 107 116 120 123 130 138
 172 180 190 201 205 211 216 217 221 229 233 235]
TRAIN: [  1   2   3   4   5

In [50]:
best_val_situ.to_csv("/share/yangjunran/soil_moisture_data/time30_origin_pair/origin_era_label/kfold8_result.csv")

In [14]:
import numpy as np
import time

t1 = np.random.randn(100000000,1)
t1 = np.asarray(t1,dtype=np.uint8)
st = time.time()
t1[np.argsort(np.argsort(t1))]
et = time.time()
print(et-st)



0.675983190536499


In [17]:
import torch

In [18]:
base_dir = r'/share/wanghongliang/Model'
model_name = 'gpt2-large/text_classification'
base_dir = base_dir + '/' + model_name
dir = base_dir + '/pytorch_model.bin'
test = torch.load(dir,map_location='cpu')

In [20]:
for k,v in test.items():
    print(type(v))

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'tor

In [1]:
output_path = "/share/wanghongliang/Model/gpt2-large/text_classification/cluster_svd1/"


In [2]:
from libKMCUDA import kmeans_cuda
from collections import OrderedDict
import pandas as pd
import numpy as np
import torch 
from copy import deepcopy
import math

In [6]:
def pixel_cluster_svd_decompress(compress_dict,output_path=False):
    model_dict = OrderedDict()
    for k,v in compress_dict.items():
        if type(v) == list:
            tmp_m = torch.mm(v[1][0],v[1][1])
            tmp_m = tmp_m.flatten()[torch.argsort(torch.argsort(v[2]))]
            tmp_m = tmp_m.reshape(v[0])
            model_dict[k] = tmp_m
        else:
            model_dict[k] = v
    if output_path:
        torch.save(model_dict,output_path+'/pytorch_model.bin')
    else:
        return model_dict

In [4]:
cdict = torch.load("/share/wanghongliang/Model/gpt2-large/text_classification/cluster_svd1/cluster_svd.bin")

In [8]:
import time
st = time.time()
pixel_cluster_svd_decompress(cdict,output_path=output_path)
et =time.time()
print(et-st)

174.60023307800293
